In [ ]:
import itertools
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
import pandas as pd
import numpy as np
import matplotlib.ticker as ticker
from sklearn import preprocessing
%matplotlib inline

In [ ]:
!wget -O loan_train.csv https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/loan_train.csv

In [ ]:
df = pd.read_csv('loan_train.csv')
df.head()

In [ ]:
df['due_date'] = pd.to_datetime(df['due_date'])
df['effective_date'] = pd.to_datetime(df['effective_date'])
df.head()

In [ ]:
df.drop('Unnamed: 0', axis = 1, inplace = True)

In [ ]:
df['loan_status'].value_counts()

In [ ]:
import seaborn as sns

bins = np.linspace(df.Principal.min(), df.Principal.max(), 10)
g = sns.FacetGrid(df, col="Gender", hue="loan_status", palette="Set1", col_wrap=2)
g.map(plt.hist, 'Principal', bins=bins, ec="k")

g.axes[-1].legend()
plt.show()

In [ ]:
bins = np.linspace(df.age.min(), df.age.max(), 10)
g = sns.FacetGrid(df, col="Gender", hue="loan_status", palette="Set1", col_wrap=2)
g.map(plt.hist, 'age', bins=bins, ec="k")

g.axes[-1].legend()
plt.show()

In [ ]:
df['dayofweek'] = df['effective_date'].dt.dayofweek
bins = np.linspace(df.dayofweek.min(), df.dayofweek.max(), 10)
g = sns.FacetGrid(df, col="Gender", hue="loan_status", palette="Set1", col_wrap=2)
g.map(plt.hist, 'dayofweek', bins=bins, ec="k")
g.axes[-1].legend()
plt.show()

In [ ]:
df['weekend'] = df['dayofweek'].apply(lambda x: 1 if (x>3)  else 0)
df.head()

In [ ]:

df.groupby(['Gender'])['loan_status'].value_counts(normalize=True)

In [ ]:
df['Gender'].replace(to_replace=['male','female'], value=[0,1],inplace=True)
df.head()

In [ ]:
df.groupby(['education'])['loan_status'].value_counts(normalize=True)

In [ ]:
df[['Principal','terms','age','Gender','education']].head()


In [ ]:
Feature = df[['Principal','terms','age','Gender','weekend']]
Feature = pd.concat([Feature,pd.get_dummies(df['education'])], axis=1)
Feature.drop(['Master or Above'], axis = 1,inplace=True)
Feature.head()

In [ ]:
X = Feature
X[0:5]

In [ ]:
y = df['loan_status'].values
y[0:5]

In [ ]:
X= preprocessing.StandardScaler().fit(X).transform(X)
X[0:5]

In [ ]:
from sklearn.model_selection import StratifiedKFold

In [ ]:
folds = StratifiedKFold(n_splits = 3)
i = 0
l = []
for train_index, test_index in folds.split(X,y):
    X_train, X_test, y_train, y_test = X[train_index], X[test_index], \
                                       y[train_index], y[test_index]
    l.append([X_train, X_test, y_train, y_test])

In [ ]:
X_train, X_test, y_train, y_test = l[2][0], l[2][1], l[2][2], l[2][3]

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics

In [ ]:
k = 2
n = KNeighborsClassifier(n_neighbors = k).fit(X_train,y_train)
npred = n.predict(X_test)
metrics.accuracy_score(y_test, npred)

In [ ]:
f =[]
g = []
for k in range(1,10):
    n = KNeighborsClassifier(n_neighbors = k).fit(X_train,y_train)
    npred = n.predict(X_test)
    #print("Train set Accuracy: ", metrics.accuracy_score(y_train, n.predict(X_train)))
    #print("Test set Accuracy: ", metrics.accuracy_score(y_test, npred), k)
    f.append(metrics.accuracy_score(y_test, npred))
    g.append(k)
plt.plot(g,f)    
print(f)

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
lpTree = DecisionTreeClassifier(criterion = 'entropy', max_depth = 5)
lpTree.fit(X_train,y_train)
predTree = lpTree.predict(X_test)

In [ ]:
print("DecisionTree's Accuracy : ", metrics.accuracy_score(y_test,predTree))

In [ ]:
from sklearn import svm

In [ ]:
clf = svm.SVC(kernel='rbf', probability = True)
clf.fit(X_train, y_train) 
ysvm = clf.predict(X_test)

In [ ]:
metrics.accuracy_score(y_test, ysvm)

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
LR = LogisticRegression(C=0.0001, solver='newton-cg').fit(X_train,y_train)


In [ ]:
yhat = LR.predict(X_test)
metrics.accuracy_score(y_test, yhat)

In [ ]:
from sklearn.metrics import jaccard_similarity_score
from sklearn.metrics import f1_score
from sklearn.metrics import log_loss

In [ ]:
Load Test set for evaluation

test_df = pd.read_csv('loan_test.csv')
test_df.head()


test_df.drop('Unnamed: 0', axis = 1, inplace = True)
test_df['due_date'] = pd.to_datetime(test_df['due_date'])
test_df['effective_date'] = pd.to_datetime(test_df['effective_date'])
test_df['dayofweek'] = test_df['effective_date'].dt.dayofweek
test_df['weekend'] = test_df['dayofweek'].apply(lambda x: 1 if (x>3)  else 0)
test_df.groupby(['Gender'])['loan_status'].value_counts(normalize=True)
test_df['Gender'].replace(to_replace=['male','female'], value=[0,1],inplace=True)
test_df.groupby(['education'])['loan_status'].value_counts(normalize=True)
Feature_test = test_df[['Principal','terms','age','Gender','weekend']]
Feature_test = pd.concat([Feature_test,pd.get_dummies(test_df['education'])], axis=1)
Feature_test.drop(['Master or Above'], axis = 1,inplace=True)
X_T = Feature_test
y_T = test_df['loan_status'].values


KNN


npred_t = n.predict(X_T)
metrics.accuracy_score(y_T, npred_t)
npred_t_prob = n.predict_proba(X_T)
f1_score(y_T, npred_t, average='weighted') 
jaccard_similarity_score(y_T, npred_t)
log_loss(y_T, npred_t_prob)


SVM

ysvm_t = clf.predict(X_T)
metrics.accuracy_score(y_T, ysvm_t)
ysvm_t_prob = clf.predict_proba(X_T)
f1_score(y_T, ysvm_t, average='weighted')
jaccard_similarity_score(y_T, ysvm_t)
log_loss(y_T, ysvm_t_prob)


Decision Tree

predTree_t = lpTree.predict(X_T)
metrics.accuracy_score(y_T, predTree_t)
predTree_t_prob = lpTree.predict_proba(X_T)
jaccard_similarity_score(y_T, predTree_t)
f1_score(y_T, predTree_t, average='weighted') 
log_loss(y_T, predTree_t_prob)


Logistic Regression

yhat_t = LR.predict(X_T)
metrics.accuracy_score(y_T, yhat_t)
yhat_t_prob = LR.predict_proba(X_T)
f1_score(y_T, yhat_t, average='weighted') 
jaccard_similarity_score(y_T, yhat_t)
log_loss(y_T, yhat_t_prob)
